In [6]:
from deepface import DeepFace
import cv2
import os
import datetime
import platform

def is_gui_available():
    return platform.system() != "Linux" or os.environ.get("DISPLAY") is not None

def create_output_dir():
    os.makedirs("result_deepface", exist_ok=True)
    return "result_deepface"

def analyze_and_draw(frame, return_infos=False):
    face_infos = []
    try:
        results = DeepFace.analyze(frame, detector_backend='retinaface', actions=["age", "gender", "emotion"], enforce_detection=False)

        # Nếu chỉ có 1 khuôn mặt, DeepFace trả về dict chứ không phải list
        if isinstance(results, dict):
            results = [results]
            
         # ➕ Sắp xếp theo vị trí từ trên xuống dưới, trái sang phải
        results.sort(key=lambda f: (f["region"].get("y", 0), f["region"].get("x", 0)))

        for idx, face in enumerate(results, start=1):
            region = face.get("region", {})
            if not region:
                continue
            x, y, w, h = region.get("x", 0), region.get("y", 0), region.get("w", 0), region.get("h", 0)
            if w == 0 or h == 0:
                continue # bỏ qua nếu không có tọa độ hợp lệ

            gender_data = face.get("gender", {})
            gender = max(gender_data, key=gender_data.get) if isinstance(gender_data, dict) else str(gender_data)
            age = str(face.get("age", "?"))
            emotion = str(face.get("dominant_emotion", "?"))

            label_number = f"#{idx}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label_number, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            face_infos.append((idx, gender, age, emotion))
    except Exception as e:
        print("Lỗi phân tích gương mặt:", e)

    if return_infos:
        return frame, face_infos
    return frame

def analyze_from_image(image_path):
    if not os.path.exists(image_path):
        print("Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    result_img, face_infos = analyze_and_draw(image, return_infos=True)

    output_dir = create_output_dir()
    filename = f"deepface_{os.path.basename(image_path)}"
    name_wo_ext = os.path.splitext(filename)[0]
    txt_filename = f"{name_wo_ext}.txt"

    output_img_path = os.path.join(output_dir, filename)
    output_txt_path = os.path.join(output_dir, txt_filename)

    # Lưu ảnh
    cv2.imwrite(output_img_path, result_img)
    print(f"\n Ảnh kết quả đã lưu tại: {output_img_path}")

    # Ghi thông tin vào file TXT
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(f"Thông tin nhận diện từ ảnh: {image_path}\n\n")
        for idx, gender, age, emotion in face_infos:
            f.write(f"#{idx}: Gender: {gender} | Age: {age} | Emotion: {emotion}\n")

    print(f" File thông tin đã lưu tại: {output_txt_path}")

    if is_gui_available():
        cv2.imshow("Kết quả nhận diện", result_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def analyze_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở webcam.")
        return

    output_dir = create_output_dir()
    analyzed_frame = None
    face_infos = []

    print(" Đang phân tích từ webcam... Nhấn 'e' hoặc 'ESC' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        try:
            frame, face_infos = analyze_and_draw(frame, return_infos=True)
            analyzed_frame = frame.copy()
        except Exception as e:
            print("Lỗi phân tích:", e)

        if is_gui_available():
            cv2.imshow("DeepFace webcam", frame)

        key = cv2.waitKey(1)
        if key == 27 or key == ord('e'):
            print("Đã thoát webcam.")
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()

    if analyzed_frame is not None:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"deepface_webcam_{timestamp}.jpg"
        txt_filename = f"deepface_webcam_{timestamp}.txt"
        output_img_path = os.path.join(output_dir, filename)
        output_txt_path = os.path.join(output_dir, txt_filename)

        cv2.imwrite(output_img_path, analyzed_frame)

        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write("Thông tin nhận diện từ webcam:\n\n")
            for idx, gender, age, emotion in face_infos:
                f.write(f"#{idx}: Gender: {gender} | Age: {age} | Emotion: {emotion}\n")

        print(f"\n Đã lưu ảnh: {output_img_path}")
        print(f" Đã lưu file thông tin: {output_txt_path}")
    else:
        print("Không có ảnh nào được lưu.")

def main():
    while True:
        print("\n=== DeepFace Face Detection ===")
        print("1. Nhận diện từ Ảnh")
        print("2. Nhận diện từ Webcam")
        print("3. Thoát chương trình")
        choice = input("Nhập lựa chọn (1/2/3): ")

        if choice == '1':
            path = input("Nhập đường dẫn ảnh: ")
            analyze_from_image(path)
        elif choice == '2':
            analyze_from_webcam()
        elif choice == '3':
            print("Kết thúc chương trình.")
            break
        else:
            print("Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()



=== DeepFace Face Detection ===
1. Nhận diện từ Ảnh
2. Nhận diện từ Webcam
3. Thoát chương trình
25-07-27 11:01:36 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: C:\Users\ACER\.deepface\weights\retinaface.h5
100%|██████████| 119M/119M [00:04<00:00, 27.3MB/s] 
Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  3.96it/s]



 Ảnh kết quả đã lưu tại: result_deepface\deepface_img4.jpg
 File thông tin đã lưu tại: result_deepface\deepface_img4.txt

=== DeepFace Face Detection ===
1. Nhận diện từ Ảnh
2. Nhận diện từ Webcam
3. Thoát chương trình
Lựa chọn không hợp lệ.

=== DeepFace Face Detection ===
1. Nhận diện từ Ảnh
2. Nhận diện từ Webcam
3. Thoát chương trình


Action: emotion: 100%|██████████| 3/3 [00:00<00:00,  3.45it/s]



 Ảnh kết quả đã lưu tại: result_deepface\deepface_person1.jpg
 File thông tin đã lưu tại: result_deepface\deepface_person1.txt

=== DeepFace Face Detection ===
1. Nhận diện từ Ảnh
2. Nhận diện từ Webcam
3. Thoát chương trình
Kết thúc chương trình.
